## 批次整理北榮資料集

In [4]:
import shutil
from pathlib import Path
import os
import re

# 設置您的路徑變數
#PATH00 = Path(r"D:\Daniel\LDCT\PREPROCESSING\Karen_work\all_data\new_dataset_0826_x\drive-download-20250826T020057Z-1-001")  
#TARGET00 = Path(r"D:\Daniel\LDCT\PREPROCESSING\Karen_work\all_data\new_dataset_0826_x\preprocessed")
#PATH0 = Path(r"D:\Daniel\LDCT\PREPROCESSING\Karen_work\all_data\new_dataset_0903_x\drive-download-20250903T033809Z-1-001")  # 替換為您的來源主路徑，例如: Path("/data/dcm_files")
#TARGET0 = Path(r"D:\Daniel\LDCT\PREPROCESSING\Karen_work\all_data\new_dataset_0903_x\preprocessed") # 替換為您的目標主路徑，例如: Path("/data/processed_dcm")
#PATH1 = Path(r"D:\Daniel\LDCT\PREPROCESSING\Karen_work\all_data\new_dataset_0905_x\北榮胸腔 CT-20250905T022723Z-1-001\北榮胸腔 CT")  
#TARGET1 = Path(r"D:\Daniel\LDCT\PREPROCESSING\Karen_work\all_data\new_dataset_0905_x\preprocessed")

PATH = Path(r"D:\Daniel\LDCT\PREPROCESSING\Karen_work\all_data\new_dataset_0910_x\北榮胸腔 CT-20250910T015644Z-1-001+002\北榮胸腔 CT")  
TARGET = Path(r"D:\Daniel\LDCT\PREPROCESSING\Karen_work\all_data\new_dataset_0910_x\preprocessed")

# 確保目標主路徑存在
TARGET.mkdir(parents=True, exist_ok=True)

# 匹配路徑結構的正規表達式
# 這個正則表達式會捕捉 caseid 和 casenumber
# 假設 caseid 和 casenumber 都是一個或多個非斜線的字元。
# 如果它們有特定的格式（例如都只有數字），您可能需要調整 `[^/]+`
path_pattern = re.compile(r"([^/]+)/([^/]+)/CT/.*\.dcm$")

print(f"--- 開始移動檔案 ---")

# 使用 rglob 遞迴地尋找所有符合模式的 .dcm 檔案
# '**/CT/*.dcm' 會尋找任何子資料夾下的 CT 資料夾中的 .dcm 檔案
for source_file in PATH.glob("**/CT/*.dcm"):
    # 取得相對於 PATH 的子路徑部分
    relative_path = source_file.relative_to(PATH).as_posix()
    
    # 使用正規表達式匹配並提取 caseid 和 casenumber
    match = path_pattern.search(relative_path)
    
    if match:
        caseid = match.group(1)
        casenumber = match.group(2)
        
        # 構建新的資料夾名稱: TARGET/{caseid}_{casenumber}/
        new_folder_name = f"{caseid}_{casenumber}"
        destination_dir = TARGET / new_folder_name
        
        # 創建新的目標資料夾 (如果不存在)
        destination_dir.mkdir(parents=True, exist_ok=True)
        
        # 構建目標檔案路徑
        # 保持原始檔案名稱
        destination_file = destination_dir / source_file.name
        
        try:
            # 移動檔案
            shutil.move(str(source_file), str(destination_file))
            # print(f"移動成功: {source_file.name} -> {destination_file}")
            
        except Exception as e:
            print(f"移動檔案 {source_file} 失敗: {e}")
            
    else:
        print(f"無法從路徑解析 caseid/casenumber: {source_file}")

print(f"--- 檔案移動完成 ---")

--- 開始移動檔案 ---
--- 檔案移動完成 ---


## 批次疊N張CT圖

In [5]:
import os
from pathlib import Path
from typing import List, Tuple, Optional, Dict
import numpy as np
import pydicom
from PIL import Image
from tqdm import tqdm

# ==== 參數設定 ====
#SRC_ROOT = Path(r"D:\Daniel\LDCT\PREPROCESSING\Karen_work\all_data\needlabel\CT_32")
SRC_ROOT = Path(r"C:\Users\ygz08\Work\LDCT")
DST_ROOT = Path(r"C:\Users\ygz08\Work\stacked")
N = 4         # 前後各取 N 張
STRIDE = 1    # 中心切片步長
DEBUG = True  # 除錯資訊開關

# ==== 函式 ====
def _read_instance_number(ds) -> Optional[int]:
    try:
        return int(getattr(ds, "InstanceNumber"))
    except Exception:
        return None

def _is_dicom_file(path: Path) -> bool:
    try:
        with open(path, "rb") as f:
            pre = f.read(132)
        return len(pre) >= 132 and pre[128:132] == b"DICM"
    except Exception:
        return False

def load_slice_info(dcm_path: Path) -> Tuple[np.ndarray, Optional[int], str, Tuple[int, int]]:
    """讀取單張 DICOM，回傳：(uint8影像, InstanceNumber, SeriesInstanceUID, shape)"""
    ds = pydicom.dcmread(str(dcm_path), force=True)
    arr = ds.pixel_array.astype(np.float32)

    slope = float(getattr(ds, "RescaleSlope", 1.0))
    intercept = float(getattr(ds, "RescaleIntercept", 0.0))
    arr = arr * slope + intercept

    low, high = np.percentile(arr, [1, 99])
    if high - low < 1e-5:
        high = low + 1.0
    arr = np.clip((arr - low) / (high - low), 0, 1.0)
    arr = (arr * 255.0).round().astype(np.uint8)

    instance_num = _read_instance_number(ds)
    series_uid = getattr(ds, "SeriesInstanceUID", "UNKNOWN_SERIES")
    return arr, instance_num, series_uid, arr.shape

def collect_dicoms_by_series(patient_dir: Path) -> Dict[Tuple[str, Tuple[int, int]], List[Tuple[np.ndarray, Optional[int]]]]:
    """依 series_uid 與 shape 分組 DICOM"""
    series_dict: Dict[Tuple[str, Tuple[int, int]], List[Tuple[np.ndarray, Optional[int]]]] = {}
    for p in patient_dir.rglob("*"):
        if p.is_file():
            if p.suffix.lower() == ".dcm" or _is_dicom_file(p):
                try:
                    img, inst, series_uid, shape = load_slice_info(p)
                    key = (series_uid, shape)
                    series_dict.setdefault(key, []).append((img, inst))
                except Exception as e:
                    print(f"[WARN] Skip {p} ({e})")
    for key in series_dict:
        series_dict[key].sort(key=lambda x: (x[1] is None, x[1] if x[1] is not None else 0))
    return series_dict

def make_mip_stack(volume: List[np.ndarray], center_idx: int, N: int) -> np.ndarray:
    start, end = center_idx - N, center_idx + N + 1
    return np.stack(volume[start:end], axis=0).max(axis=0)

def save_jpeg(img: np.ndarray, out_path: Path):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    Image.fromarray(img).save(str(out_path), format="JPEG", quality=95)

# ==== 主程式 ====
assert SRC_ROOT.exists(), f"來源不存在：{SRC_ROOT}"
DST_ROOT.mkdir(parents=True, exist_ok=True)

patient_dirs = [p for p in SRC_ROOT.iterdir() if p.is_dir()]
if DEBUG:
    print(f"[INFO] src = {SRC_ROOT}")
    print(f"[INFO] dst = {DST_ROOT}")
    print(f"[INFO] N = {N}, stride = {STRIDE}")
    print(f"[INFO] Found {len(patient_dirs)} patient folders.")

total_inputs = 0
total_outputs = 0

for pdir in tqdm(sorted(patient_dirs), desc="Patients"):
    series_dict = collect_dicoms_by_series(pdir)
    if DEBUG:
        print(f"\n[INFO] {pdir.name}: found {len(series_dict)} series")

    for (series_uid, shape), items in series_dict.items():
        volume = [img for img, _ in items]
        total_inputs += len(volume)

        if len(volume) < 2 * N + 1:
            print(f"[WARN] {pdir.name} | Series {series_uid} shape={shape} has only {len(volume)} slices, skip.")
            continue

        # **直接輸出在病人資料夾，不建立 series 子資料夾**
        out_dir = DST_ROOT / pdir.name
        count_out = 0
        for i in range(N, len(volume) - N, STRIDE):
            mip = make_mip_stack(volume, i, N)
            # **檔名前加上 series_uid 避免覆蓋**
            out_name = f"{series_uid}_{i:04d}_mip_N{N}.jpg"
            save_jpeg(mip, out_dir / out_name)
            count_out += 1

        total_outputs += count_out
        print(f"[OK] {pdir.name} | Series {series_uid} shape={shape}: input_slices={len(volume)}, outputs={count_out}")

print(f"\nDone. patients={len(patient_dirs)}, input_slices={total_inputs}, outputs={total_outputs}, N={N}, stride={STRIDE}")


[INFO] src = C:\Users\ygz08\Work\LDCT
[INFO] dst = C:\Users\ygz08\Work\stacked
[INFO] N = 4, stride = 1
[INFO] Found 97 patient folders.


Patients:   0%|          | 0/97 [00:00<?, ?it/s]


[INFO] 10203514_D7K32J8: found 1 series


Patients:   1%|          | 1/97 [00:13<21:02, 13.15s/it]

[OK] 10203514_D7K32J8 | Series 1.2.392.200036.9116.2.6.1.41014.3167532101.1753157426.377207 shape=(512, 512): input_slices=311, outputs=303

[INFO] 10449360_D7J83AN: found 1 series


...

[OK] 9720030_D7K2P3D | Series 1.2.392.200036.9116.2.6.1.41014.3167532101.1753081558.731502 shape=(512, 512): input_slices=356, outputs=348

[INFO] 97784015_D7K316K: found 1 series


Patients:  99%|█████████▉| 96/97 [23:28<00:15, 15.23s/it]

[OK] 97784015_D7K316K | Series 1.3.12.2.1107.5.1.4.74304.30000025080723575682200073800 shape=(512, 512): input_slices=350, outputs=342

[INFO] 99114585_D7J84E8: found 1 series


Patients: 100%|██████████| 97/97 [23:45<00:00, 14.70s/it]

[OK] 99114585_D7J84E8 | Series 1.3.12.2.1107.5.1.4.74304.30000025061223452152600043337 shape=(512, 512): input_slices=367, outputs=359

Done. patients=97, input_slices=30858, outputs=30082, N=4, stride=1


## 批次壓縮CT資料夾

In [6]:
import shutil
from pathlib import Path
from tqdm import tqdm

# 來源與輸出資料夾
SRC_ROOT = Path(r"C:\Users\ygz08\Work\stacked")
DST_ROOT = SRC_ROOT.parent / "CT_zips"
DST_ROOT.mkdir(parents=True, exist_ok=True)

for folder in tqdm(sorted(SRC_ROOT.iterdir()), desc="Zipping"):
    if folder.is_dir():
        zip_path = DST_ROOT / folder.name  # 壓縮檔路徑（不含副檔名）
        shutil.make_archive(str(zip_path), 'zip', root_dir=folder)
        print(f"[OK] {folder.name} -> {zip_path}.zip")

print(f"壓縮完成！共 {len(list(SRC_ROOT.iterdir()))} 個資料夾，輸出到：{DST_ROOT}")


Zipping:   1%|          | 1/97 [00:02<03:29,  2.18s/it]

[OK] 10203514_D7K32J8 -> C:\Users\ygz08\Work\CT_zips\10203514_D7K32J8.zip


Zipping:   2%|▏         | 2/97 [00:04<03:36,  2.27s/it]

[OK] 10449360_D7J83AN -> C:\Users\ygz08\Work\CT_zips\10449360_D7J83AN.zip


Zipping:   3%|▎         | 3/97 [00:07<03:46,  2.41s/it]

...

[OK] 9720030_D7K2P3D -> C:\Users\ygz08\Work\CT_zips\9720030_D7K2P3D.zip


Zipping:  99%|█████████▉| 96/97 [03:50<00:02,  2.40s/it]

[OK] 97784015_D7K316K -> C:\Users\ygz08\Work\CT_zips\97784015_D7K316K.zip


Zipping: 100%|██████████| 97/97 [03:53<00:00,  2.41s/it]

[OK] 99114585_D7J84E8 -> C:\Users\ygz08\Work\CT_zips\99114585_D7J84E8.zip
壓縮完成！共 97 個資料夾，輸出到：C:\Users\ygz08\Work\CT_zips


In [ ]:
!pip install cvat-cli


  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/780.6 kB ? eta -:--:--
   ---------------------------------------- 780.6/780.6 kB 11.0 MB/s  0:00:00
Using cached attrs-25.4.0-py3-none-any.whl (67 kB)

   ------------- -------------------------- 1/3 [cvat-sdk]
   ------------- -------------------------- 1/3 [cvat-sdk]
   ------------- -------------------------- 1/3 [cvat-sdk]
   ------------- -------------------------- 1/3 [cvat-sdk]
   -------------------------- ------------- 2/3 [cvat-cli]
   ---------------------------------------- 3/3 [cvat-cli]

Note: you may need to restart the kernel to use updated packages.


## 批次上傳檔案至CVAT 切外網****

In [12]:
"""
Batch create CVAT tasks from ZIPs (one ZIP -> one Task) into a given Project.

Requirements:
    pip install requests

Usage:
    python cvat_batch_create_from_zips.py
"""

import os
import time
import requests
from urllib.parse import quote

# ==== CONFIG ====
SERVER      = "http://cvat.muen.tw:8080/"   # e.g. "http://10.0.0.5:8080"
USERNAME    = "muen-label-1"
PASSWORD    = "pYLHXrm82M6K22Y"
PROJECT_ID  = 22                 # your project id (already has labels)
ZIP_DIR     = r"C:\Users\ygz08\Work\CT_zips"      # folder containing *.zip
IMAGE_QUALITY = 70                     # 0~100, higher = better quality
USE_ZIP_CHUNKS = True                  # True is safer for large zips
USE_CACHE      = True                  # server-side cache if available
FRAME_STEP     = 1                     # sample every Nth frame (for videos); images unaffected
RETRY          = 3                     # upload retries
RETRY_SLEEP    = 5                     # seconds between retries
TIMEOUT        = 180                    # per-request timeout seconds
# ==============

sess = requests.Session()
sess.auth = (USERNAME, PASSWORD)

def api_get(path, **kw):
    return sess.get(f"{SERVER}{path}", timeout=TIMEOUT, **kw)

def api_post(path, **kw):
    return sess.post(f"{SERVER}{path}", timeout=TIMEOUT, **kw)

def ensure_project_has_labels(project_id: int):
    r = api_get(f"/api/projects/{project_id}")
    r.raise_for_status()
    pj = r.json()
    labels = pj.get("labels", [])
    if not labels:
        raise RuntimeError(f"Project #{project_id} has NO labels. Add labels first.")
    return pj

def find_task_by_name(name: str):
    # CVAT supports search with ?search=, but not always exact; we double-check.
    r = api_get(f"/api/tasks?search={quote(name)}")
    r.raise_for_status()
    for t in r.json().get("results", []):
        if t.get("name") == name:
            return t
    return None

def create_task(name: str, project_id: int) -> int:
    payload = {
        "name": name,
        "project_id": project_id,
    }
    r = api_post("/api/tasks", json=payload)
    r.raise_for_status()
    return r.json()["id"]

def upload_zip(task_id: int, zip_path: str):
    # Build form fields accepted by /api/tasks/{id}/data
    data = {
        "image_quality": str(IMAGE_QUALITY),
        "frame_step": str(FRAME_STEP),
        "use_zip_chunks": "true" if USE_ZIP_CHUNKS else "false",
        "use_cache": "true" if USE_CACHE else "false",
    }
    # Important: name the part as client_files[0]
    with open(zip_path, "rb") as fh:
        files = {
            "client_files[0]": (os.path.basename(zip_path), fh, "application/zip")
        }
        r = api_post(f"/api/tasks/{task_id}/data", data=data, files=files)
        r.raise_for_status()

def main():
    ensure_project_has_labels(PROJECT_ID)
    zips = [f for f in os.listdir(ZIP_DIR) if f.lower().endswith(".zip")]
    zips.sort()
    if not zips:
        print(f"No zip files found in: {ZIP_DIR}")
        return

    for z in zips:
        zip_path = os.path.join(ZIP_DIR, z)
        task_name = os.path.splitext(z)[0]
        try:
            existing = find_task_by_name(task_name)
            if existing:
                print(f"[SKIP] Task already exists: '{task_name}' (id={existing['id']})")
                continue

            print(f"[CREATE] Task: {task_name}")
            task_id = create_task(task_name, PROJECT_ID)
            print(f"  -> Task ID: {task_id}; uploading data: {z}")

            # retry upload a few times
            for attempt in range(1, RETRY + 1):
                try:
                    upload_zip(task_id, zip_path)
                    print("  -> Upload OK")
                    break
                except requests.RequestException as e:
                    if attempt >= RETRY:
                        raise
                    print(f"  !! Upload failed (attempt {attempt}/{RETRY}): {e}")
                    time.sleep(RETRY_SLEEP)

        except Exception as e:
                    print(f"[ERROR] '{z}': {e}")

if __name__ == "__main__":
    main()

[CREATE] Task: 10203514_D7K32J8
  -> Task ID: 276; uploading data: 10203514_D7K32J8.zip
  -> Upload OK
[CREATE] Task: 10449360_D7J83AN
  -> Task ID: 277; uploading data: 10449360_D7J83AN.zip
  -> Upload OK
[CREATE] Task: 10670766_D72388B
  -> Task ID: 278; uploading data: 10670766_D72388B.zip
  -> Upload OK
[CREATE] Task: 10901001_D7K5H19
  -> Task ID: 279; uploading data: 10901001_D7K5H19.zip
  -> Upload OK
[CREATE] Task: 11016499_D7K5GLM
  -> Task ID: 280; uploading data: 11016499_D7K5GLM.zip
  -> Upload OK
[CREATE] Task: 12753256_D75DDP4
  -> Task ID: 281; uploading data: 12753256_D75DDP4.zip
  -> Upload OK
[CREATE] Task: 12962803_D735KKC
  -> Task ID: 282; uploading data: 12962803_D735KKC.zip
  -> Upload OK
[CREATE] Task: 13726467_D742998
  -> Task ID: 283; uploading data: 13726467_D742998.zip
  -> Upload OK
[CREATE] Task: 14093855_D7K57H6
  -> Task ID: 284; uploading data: 14093855_D7K57H6.zip
  -> Upload OK
[CREATE] Task: 14294814_D7K53NC
  -> Task ID: 285; uploading data: 142948